In [1]:
import getpass
import os
import json

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [2]:

from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

loader = PyPDFLoader("./chess.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [11]:
text = format_docs(docs)

import tiktoken
model = "gpt-4"
encoding = tiktoken.encoding_for_model(model)

tokens = encoding.encode(text)
token_count = len(tokens)

print(token_count)

5775


In [3]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


/home/amogha/.local/lib/python3.10/site-packages/langsmith/client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [4]:
with open("scene.json") as scene_file, open("scene-validator.json") as validator_file:
    scene_data = json.load(scene_file)
    validator_data = json.load(validator_file)


def generate_question_for_field(field_name, validation_rules):
    typeof = validation_rules.get("typeof", "text")
    query = validation_rules.get("query", f"Please provide the value for {field_name}")
    comment = validation_rules.get("%comment%", "")
    
    question = f"{query} (Expected type: {typeof})?"
    if comment:
        question += f" Note: {comment}"
    
    if typeof == "object":
        question = f"How many {query}? Please provide details for each. {comment}"
    
    return question

questions = {
    field: generate_question_for_field(field, rules)
    for field, rules in validator_data.items()
}

In [5]:
print(questions)

{'_scenename': 'Please provide the value for _scenename (Expected type: string)? Note: Mention your VR Scene name here', '_sid': 'Please provide the value for _sid (Expected type: string)? Note: A fillable Unique Identifier of the scene', '_slabel': 'Please provide the value for _slabel (Expected type: string)? Note: A fillable Optional text field for scene description in 200 words', '#pid': 'Please provide the value for #pid (Expected type: string)? Note: Unique identifier for playarea in the VR Scene', '#length_playarea': 'Please provide the value for #length_playarea (Expected type: string)? Note: Length dimension of the playarea and it is mandatory value tag', '#breadth_playarea': 'Please provide the value for #breadth_playarea (Expected type: string)? Note: Breadth dimension of the playarea and it is mandatory value tag', '#height_playarea': 'Please provide the value for #height_playarea (Expected type: string)? Note: Height dimension of the playarea and it is mandatory value tag'

In [19]:
token_count = 0
for field, question in questions.items():
    tokens = encoding.encode(question)
    token_count += len(tokens)

print(token_count)

1615


In [ ]:
prompt_text = 'You are tasked with generating the VR requirements for a chess game scene. Based on the given JSON structure, you should fill in the required fields by answering questions related to the environment setup, user interactions, game mechanics, user interface, and immersion enhancements. Your responses should provide detailed specifications that will be used to populate the scene configuration.'

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

responses = []
for field, question in questions.items():
    print(f"Filling field '{field}'")
    full_prompt = f"{prompt_text}\n\nQuestion: {question}"
    response = rag_chain.invoke(full_prompt)
    print(f"Answer: {response}")
    responses.append(response)


Filling field '_scenename'
Answer: The value for _scenename is "Chess VR Scene".
Filling field '_sid'
Answer: I don't know.
Filling field '_slabel'
Answer: The chess game scene is set on a classic 8x8 chessboard with alternating light and dark squares, typically in white and brown or white and green. Each player controls a set of 16 pieces, including a king, queen, rooks, bishops, knights, and pawns, arranged in a specific formation on their respective ranks. The immersive environment captures the essence of competitive chess, reflecting its rich history and strategic depth.
Filling field '#pid'
Answer: I don't know.
Filling field '#length_playarea'
Answer: I don't know.
Filling field '#breadth_playarea'
Answer: The value for #breadth_playarea should be "8 squares" to reflect the 8 columns in the chess game setup.
Filling field '#height_playarea'
Answer: The value for #height_playarea should be set to "1.5 meters" to ensure an adequate vertical space for player interactions in the VR c

In [ ]:
prompt_text = 'You are tasked with generating the VR requirements for a chess game scene. Based on the given JSON structure, you should fill in the required fields by answering questions related to the environment setup, user interactions, game mechanics, user interface, and immersion enhancements. Keep your responses brief.'

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

responses = []
for field, question in questions.items():
    print(f"Filling field '{field}'")
    full_prompt = f"{prompt_text}\n\nQuestion: {question}"
    response = rag_chain.invoke(full_prompt)
    print(f"Answer: {response}")
    responses.append(response)


Filling field '_scenename'
Answer: The VR scene name should be "Virtual Chess Arena."
Filling field '_sid'
Answer: I don't know.
Filling field '_slabel'
Answer: The chess game scene is set on an 8x8 grid board featuring alternating light and dark squares, traditionally in colors like white and brown or green. Each player controls 16 pieces: one king, one queen, two rooks, two bishops, two knights, and eight pawns, arranged according to established placement rules. The immersive environment allows players to engage in strategic gameplay, reflecting the rich history and cultural significance of chess.
Filling field '#pid'
Answer: I don't know.
Filling field '#length_playarea'
Answer: I don't know.
Filling field '#breadth_playarea'
Answer: The value for #breadth_playarea is "8 squares".
Filling field '#height_playarea'
Answer: I don't know.
Filling field '#x_scenecenter'
Answer: The value for #x_scenecenter should be 0, as it represents the x-axis value of the scene center in a standard c

In [21]:
print(rag_chain.invoke('Tell me about chess'))

Chess is a two-player board game played on an 8×8 grid, involving strategic moves of sixteen pieces per player, including a king and queen. It has roots in the seventh-century Indian game chaturanga and evolved through Persia to Europe, with modern rules established by the end of the 15th century. Today, chess is governed internationally by FIDE and is one of the most popular games worldwide, with a rich body of theory and competitive history.
